In [ ]:
import os
import json
import requests
import xml.etree.ElementTree as ET
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime, timedelta
from openpyxl import Workbook, load_workbook

# ファイル名設定
keyword_file = "search_keywords.json"
result_file = "PubMed_results.xlsx"
notified_ids_file = "notified_ids.json"

# 環境変数から機密情報を取得
api_key = os.getenv("PUBMED_API_KEY")
sender_email = os.getenv("SENDER_EMAIL")
sender_password = os.getenv("SENDER_PASSWORD")
recipient = os.getenv("RECIPIENT")

# キーワードの保存と読み込み
def save_keywords(keywords):
    with open(keyword_file, "w") as f:
        json.dump(keywords, f)

def load_keywords():
    try:
        with open(keyword_file, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return []

# 過去の通知済みIDを管理
def load_notified_ids():
    try:
        with open(notified_ids_file, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return []

def save_notified_ids(ids):
    with open(notified_ids_file, "w") as f:
        json.dump(ids, f)

# PubMed API検索
def search_pubmed(query, start_date, end_date, api_key):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    date_range = f"{start_date}:{end_date}[dp]"
    full_query = f"{query} AND {date_range}"
    params = {
        "db": "pubmed",
        "term": full_query,
        "retmode": "json",
        "sort": "date",
        "retmax": 10,
        "api_key": api_key
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    return response.json()["esearchresult"]["idlist"]

# PubMed APIで論文詳細を取得
def fetch_abstracts(ids, api_key):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {
        "db": "pubmed",
        "id": ",".join(ids),
        "retmode": "xml",
        "rettype": "abstract",
        "api_key": api_key
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    return response.text

# XML解析関数
def parse_pubmed_data(xml_data):
    root = ET.fromstring(xml_data)
    articles = []
    for article in root.findall(".//PubmedArticle"):
        pmid = article.find(".//PMID").text if article.find(".//PMID") is not None else "No PMID"
        title = article.find(".//ArticleTitle").text if article.find(".//ArticleTitle") is not None else "No Title"

        # TitleがNoneまたは短い場合の警告追加
        if not title or len(title) < 10:
            title = (title or "No Title") + " [Warning: Title might be incomplete]"

        # Abstract処理
        abstract_parts = [abstract.text for abstract in article.findall(".//AbstractText") if abstract.text]
        abstract = " ".join(abstract_parts).strip() if abstract_parts else "No Abstract"
        if len(abstract) < 50:
            abstract += " [Warning: Abstract might be incomplete]"

        # Author情報の取得
        authors = article.findall(".//Author")
        first_author = "No First Author"
        first_affiliation = "No First Author Affiliation"
        last_author = "No Last Author"
        last_affiliation = "No Last Author Affiliation"

        if authors:
            first = authors[0]
            if first.find("ForeName") is not None and first.find("LastName") is not None:
                first_author = f"{first.find('ForeName').text} {first.find('LastName').text}"
            if first.find(".//AffiliationInfo/Affiliation") is not None:
                first_affiliation = first.find(".//AffiliationInfo/Affiliation").text

            last = authors[-1]
            if last.find("ForeName") is not None and last.find("LastName") is not None:
                last_author = f"{last.find('ForeName').text} {last.find('LastName').text}"
            if last.find(".//AffiliationInfo/Affiliation") is not None:
                last_affiliation = last.find(".//AffiliationInfo/Affiliation").text

        author_list = ", ".join(
            f"{author.find('ForeName').text} {author.find('LastName').text}"
            for author in authors
            if author.find("ForeName") is not None and author.find("LastName") is not None
        ) if authors else "No Authors"

        journal = article.find(".//Title").text if article.find(".//Title") is not None else "No Journal"
        pub_date = article.find(".//PubDate/Year").text if article.find(".//PubDate/Year") is not None else "No Date"
        link = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"

        articles.append({
            "pmid": pmid,
            "title": title,
            "abstract": abstract,
            "first_author": first_author,
            "first_affiliation": first_affiliation,
            "last_author": last_author,
            "last_affiliation": last_affiliation,
            "authors": author_list,
            "journal": journal,
            "pub_date": pub_date,
            "link": link,
        })
    return articles


# メール本文のHTMLフォーマット
def format_html_email(articles, keyword):
    content = f"<html><body><h2>Results for {keyword}</h2><hr>"
    for article in articles:
        content += f"""
        <p><strong>Title:</strong> {article['title']}</p>
        <p><strong>PMID:</strong> <a href="{article['link']}">{article['pmid']}</a></p>
        <p><strong>Abstract:</strong> {article['abstract']}</p>
        <p><strong>First Author:</strong> {article['first_author']} ({article['first_affiliation']})</p>
        <p><strong>Last Author:</strong> {article['last_author']} ({article['last_affiliation']})</p>
        <p><strong>Authors:</strong> {article['authors']}</p>
        <p><strong>Journal:</strong> {article['journal']} ({article['pub_date']})</p>
        <hr>
        """
    content += "</body></html>"
    return content

# メール送信
def send_email(subject, html_content):
    msg = MIMEMultipart("alternative")
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = recipient
    msg.attach(MIMEText(html_content, "html"))
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient, msg.as_string())

# Excel保存
def save_to_excel(articles, sheet_name):
    if os.path.exists(result_file):
        workbook = load_workbook(result_file)
    else:
        workbook = Workbook()
    if sheet_name in workbook.sheetnames:
        del workbook[sheet_name]
    sheet = workbook.create_sheet(sheet_name)
    sheet.append(["PMID", "Title", "Abstract", "First Author", "First Affiliation", "Last Author", "Last Affiliation", "Authors", "Journal", "Publication Date", "Link"])
    for article in articles:
        sheet.append([article["pmid"], article["title"], article["abstract"], article["first_author"], article["first_affiliation"], article["last_author"], article["last_affiliation"], article["authors"], article["journal"], article["pub_date"], article["link"]])
    workbook.save(result_file)

# 新しい記事のみフィルタリング
def filter_new_articles(articles, notified_ids):
    return [article for article in articles if article["pmid"] not in notified_ids]

# メイン処理
def main(keywords):
    notified_ids = load_notified_ids()
    start_date = (datetime.now() - timedelta(days=1)).strftime("%Y/%m/%d")
    end_date = datetime.now().strftime("%Y/%m/%d")

    for keyword in keywords:
        ids = search_pubmed(keyword, start_date, end_date, api_key)
        if ids:
            xml_data = fetch_abstracts(ids, api_key)
            articles = parse_pubmed_data(xml_data)
            new_articles = filter_new_articles(articles, notified_ids)

            if new_articles:
                save_to_excel(new_articles, sheet_name=keyword)
                html_content = format_html_email(new_articles, keyword)
                send_email(subject=f"PubMed Results for {keyword}", html_content=html_content)
                notified_ids.extend([article["pmid"] for article in new_articles])
                save_notified_ids(notified_ids)
            else:
                print(f"No new articles for keyword: {keyword}")
        else:
            print(f"No results for keyword: {keyword}")

# 実行
keywords = load_keywords()
if keywords:
    main(keywords)
else:
    print("No keywords provided. Please add keywords to 'search_keywords.json'.")
